<i>The Battle of Neighborhoods (Week 2) by Dr K.R.S Dulanjalee</i>

# <center><span style="color:green">Segmenting and Clustering Neighborhoods in Toronto</span></center>

## 1. <span style="color:orange">Import relevant libraries</span>

In [30]:
import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

import pylab as pl
import seaborn as sns
import scipy.optimize as opt
%matplotlib notebook
%matplotlib inline


import json 

!pip install geopy  
from geopy.geocoders import Nominatim

import requests 
from pandas.io.json import json_normalize 

from sklearn.cluster import KMeans

!pip install folium 
import folium 

## 2. <span style="color:orange">Load Data and make a Data Frame</span>

In [31]:
# The code was removed by Watson Studio for sharing.

,Index,Postal Codes,Latitudes,Longitudes,Borough
0,1,V0A1K1,50.339434,-115.858121,Fairmont Hot Springs
1,2,V0B0V0,49.218994,-122.949596,Burnaby
2,3,V0C2B0,56.247431,-120.879999,Fort St. John
3,4,V0E2Z7,50.711182,-119.261332,Salmon Arm
4,6,V0H0H0,49.160556,-123.114890,Richmond


In [32]:
BC_df.shape

(278, 5)

In [33]:
BC_df.columns

Index(['Index', 'Postal Codes', 'Latitudes', 'Longitudes', 'Borough'], dtype='object')

In [34]:
BC_df = BC_df.drop(['Index'], axis = 1) 
BC_df.head()

,Postal Codes,Latitudes,Longitudes,Borough
0,V0A1K1,50.339434,-115.858121,Fairmont Hot Springs
1,V0B0V0,49.218994,-122.949596,Burnaby
2,V0C2B0,56.247431,-120.879999,Fort St. John
3,V0E2Z7,50.711182,-119.261332,Salmon Arm
4,V0H0H0,49.160556,-123.114890,Richmond


In [35]:
A = BC_df[BC_df['Borough'] == 'Campbell River']
A

,Postal Codes,Latitudes,Longitudes,Borough
263,V9H0E6,49.960339,-125.236511,Campbell River
264,V9H0E7,49.961222,-125.238884,Campbell River
270,V9W0E9,45.730295,-76.603025,Campbell River
271,V9W3S4,49.906397,-125.195014,Campbell River


In [36]:
BC_df = BC_df.drop([263, 264, 270,271])
BC_df.tail(30)

,Postal Codes,Latitudes,Longitudes,Borough
244,V7X0P4,49.186800,-123.132399,Richmond
245,V7X1J1,49.286137,-123.119745,Vancouver
246,V7X1L7,49.286136,-123.119745,Vancouver
247,V8A0C7,49.886060,-124.544084,Powell River
248,V8B0E3,49.732770,-123.141706,Squamish
249,V8E0Y0,50.120881,-122.936620,Whistler
250,V8J4N7,54.310921,-130.304363,Prince Rupert
251,V8L0H9,48.411411,-123.495567,Colwood
252,V8S0C3,49.699478,-123.154697,Squamish
253,V8T0E1,48.448495,-123.336286,Victoria


In [37]:
B = BC_df[BC_df['Borough'] == 'Campbell River']
B

,Postal Codes,Latitudes,Longitudes,Borough


In [38]:
BC_df.dtypes

Postal Codes     object
Latitudes       float64
Longitudes      float64
Borough          object
dtype: object

## 3. <span style="color:orange">Visualization</span>

In [39]:
address = 'British Columbia'

geolocator = Nominatim(user_agent="British Columbia_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of British Columbia are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of British Columbia are 55.001251, -125.002441.


In [40]:
map_BritishColumbia = folium.Map(location=[latitude, longitude], zoom_start=5)

for lat, lng, borough in zip(BC_df['Latitudes'], BC_df['Longitudes'], BC_df['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_BritishColumbia)  
    
map_BritishColumbia

## 4. <span style="color:orange">Segment and cluster only the neighborhoods in Surrey</span>

In [41]:
surrey_data = BC_df[BC_df['Borough'] == 'Surrey'].reset_index(drop=True)
surrey_data.head()

,Postal Codes,Latitudes,Longitudes,Borough
0,V2S0Z5,49.186697,-122.809079,Surrey
1,V2S9J2,49.107889,-122.807934,Surrey
2,V2W5Y4,49.144302,-122.845344,Surrey
3,V3A8X9,49.043329,-122.784200,Surrey
4,V3E6Y6,49.156208,-122.852509,Surrey


In [42]:
surrey_data.shape

(41, 4)

In [48]:
map_surrey= folium.Map(location=[latitude, longitude], zoom_start=20)


for lat, lng, label in zip(surrey_data['Latitudes'], surrey_data['Longitudes'], surrey_data['Postal Codes']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_surrey)  
    
map_surrey